In [1]:
import csv
import numpy as np

NUMBER_OF_PREVIOUS_PRICES = 10

years = []
months = []
days = []
prices = []
temperature = []

def readSpotPrices():
    with open('elspotprices_train.csv', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
        for row in spamreader:
            date = row[0].split("/")
            years.append(date[2]);
            months.append(date[1]);
            days.append(date[0]);
            prices.append(float(row[1].replace(',','.')))
        
def readTemperature(): 
    with open('temperature_train.csv', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
        for row in spamreader:
            oslo = float(row[0])
            bergen = float(row[1])
            trondheim = float(row[2])
            tromso = float(row[3])
            temperature.append([oslo, bergen,trondheim, tromso])
        
def getPreviousPricesFromIndex(i):
    previous_prices = []
    for x in range(1, NUMBER_OF_PREVIOUS_PRICES+1):
        previous_prices.append(prices[i-x])
    return reversed(previous_prices)

def generateRow(index):
    row = []
    previous_prices = getPreviousPricesFromIndex(index);
    row.append(years[index])
    row.append(months[index])
    row.append(days[index])
    row.extend(temperature[index])
    row.extend(previous_prices)
    row.append(prices[index])
    row.append(prices[index+1])
    return row


def generateNormalizedRow(index):
    row = []
    previous_prices = getPreviousPricesFromIndex(index);
    
    year = (float(years[index])-13)/(16-13)*2 -1;
    month = float(months[index])/6-1
    day = float(days[index])/31*2-1
    
    maxprice = np.amax(prices)
    minprice = np.amin(prices)
    price = (prices[index] - minprice)/(maxprice-minprice)*2-1
    next_price = (prices[index+1] - minprice)/(maxprice-minprice)*2-1
    previous_prices_scaled =[]
    for pp in previous_prices:
        previous_prices_scaled.append((pp - minprice)/(maxprice-minprice)*2-1)
    
    lokasjoner = temperature[index]
    maxtemperature = np.amax(temperature)
    mintemperature = np.amin(temperature)
    lokasjoner_scaled = []
    for t in lokasjoner:
           lokasjoner_scaled.append((t - mintemperature)/(maxtemperature-mintemperature)*2 -1)
    
    row.append(year)
    row.append(month)
    row.append(day)
    row.extend(lokasjoner_scaled)
    row.extend(previous_prices_scaled)
    row.append(price)
    row.append(next_price)
    return row

def writeDataSet(name, start, end, writelabel):
    with open(name, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if writelabel: 
            writer.writerow(["year", "month", "day", 
                            "temperature oslo", "temperature bergen", "temperature trondheim", "temperature tromso",
                            "10 days ago", "9 days ago","8 days ago", "7 days ago", "6 days ago", "5 days ago",
                            "4 days ago", "3 days ago", "2 days ago", "1 day ago", "today", "tomorrow"])
        for x in range(NUMBER_OF_PREVIOUS_PRICES, NUMBER_OF_ROWS):
            row = generateRow(x)
            writer.writerow(row)
                
                
def writeDataSetScaled(name,start, end, writelabel):
    with open(name, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        if writelabel: 
            writer.writerow(["year", "month", "day", 
                            "temperature oslo", "temperature bergen", "temperature trondheim", "temperature tromso",
                            "10 days ago", "9 days ago","8 days ago", "7 days ago", "6 days ago", "5 days ago",
                            "4 days ago", "3 days ago", "2 days ago", "1 day ago", "today", "tomorrow"])
        for x in range(start,end ):
            row = generateNormalizedRow(x)
            writer.writerow(row)
                
            
readSpotPrices()
readTemperature()
NUMBER_OF_ROWS= len(years)-1
print NUMBER_OF_ROWS
print len(temperature)

writeDataSetScaled("train_scaled.csv", NUMBER_OF_PREVIOUS_PRICES,NUMBER_OF_ROWS, False )
writeDataSet("train.csv", NUMBER_OF_PREVIOUS_PRICES,NUMBER_OF_ROWS,False )
writeDataSetScaled("train_scaled_label.csv", NUMBER_OF_PREVIOUS_PRICES,NUMBER_OF_ROWS, True )
writeDataSet("train_label.csv", NUMBER_OF_PREVIOUS_PRICES,NUMBER_OF_ROWS,True )

2099
2099
